In [1]:
from __future__ import print_function, division
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import *
from keras.layers import Conv2D, UpSampling2D
from keras.layers import MaxPooling2D
from keras import *
from keras.models import Model,Sequential
from keras.utils import to_categorical
import keras.backend as k

In [2]:
pip install tensorflow

In [3]:
from keras.datasets import cifar10


In [4]:
!mkdir inpainting_images

In [5]:
!mkdir saved_model

In [6]:
import numpy as np
from scipy.linalg import sqrtm
from sklearn.metrics import pairwise_distances
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model

In [7]:
pip install scikit-image

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, Flatten, Dense, UpSampling2D, Activation, Dropout
from keras.models import Model, Sequential

from keras.datasets import cifar10
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from skimage.transform import resize

In [9]:
from tensorflow.keras.optimizers import legacy

In [10]:
class Inpainting():
  def __init__(self):
    self.img_rows = 32
    self.img_cols = 32
    self.channels = 3
    self.mask_height = 8
    self.mask_width = 8
    self.num_classes = 2
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.missing_image = (self.mask_height,self.mask_width,self.channels)
    optimizer = legacy.Adam(learning_rate=0.0002, beta_1=0.5)
    self.discriminator=self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

    self.generator=self.build_generator()

    masked_img = Input(shape=self.img_shape)
    generated_img = self.generator(masked_img)
    self.discriminator.trainable = False
    valid = self.discriminator(generated_img)
    self.combined = Model(masked_img, [generated_img,valid])
    self.combined.compile(loss=['mse','binary_crossentropy'],loss_weights= [0.999,0.001], optimizer=optimizer)

    self.inception_model = InceptionV3(include_top=False, pooling='avg', input_shape = (299,299,3))
  def build_generator(self):
    #encoder
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, strides =2, padding='same', input_shape=self.img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(64, kernel_size = 3, strides =2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size = 3, strides =2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(512, kernel_size = 1, strides =2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    #Decoder
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(self.channels, kernel_size = 3, padding='same'))
    model.add(Activation('tanh'))

    model.summary()
    masked_img = Input(shape=self.img_shape)
    generated_img = model(masked_img)
    return (Model(masked_img, generated_img))


  def build_discriminator(self):
    model = Sequential()
    model.add(Conv2D(64, kernel_size = 3, strides =2, padding='same', input_shape=self.missing_image))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size = 3, strides =2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size = 3, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    img = Input(shape=self.missing_image)
    validity = model(img)
    return (Model(img, validity))

  def mask_random(self,imgs):
    y1 = np.random.randint(0,self.img_rows-self.mask_height, imgs.shape[0])
    y2 = y1 + self.mask_height
    x1 = np.random.randint(0,self.img_rows-self.mask_width, imgs.shape[0])
    x2 = x1 + self.mask_width
    masked_imgs = np.empty_like(imgs)
    missing_parts = np.empty(shape=(imgs.shape[0],self.mask_height,self.mask_width,self.channels))

    for i, img in enumerate(imgs):
      masked_img = img.copy()
      _y1,_y2,_x1,_x2=y1[i],y2[i],x1[i],x2[i]
      missing_parts[i] = masked_img[_y1:_y2,_x1:_x2,:].copy()
      masked_img[_y1:_y2,_x1:_x2,:]=0
      masked_imgs[i] = masked_img

    return (masked_imgs,missing_parts,(y1,y2,x1,x2))
  def calculate_fid(self, real_images, fake_images):

        real_images = np.array([resize(image, (299, 299, 3), anti_aliasing=True) for image in real_images])
        fake_images = np.array([resize(image, (299, 299, 3), anti_aliasing=True) for image in fake_images])

        real_images = preprocess_input(real_images)
        fake_images = preprocess_input(fake_images)

        act1 = self.inception_model.predict(real_images)
        act2 = self.inception_model.predict(fake_images)

        mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
        mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

        ssdiff = np.sum((mu1 - mu2)**2.0)

        covmean = sqrtm(sigma1.dot(sigma2))

        if np.iscomplexobj(covmean):
            covmean = covmean.real

        fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
        return fid

  def train(self,epochs,batch_size=128,sample_interval=50):
    (X_train, y_train), (_, _) = cifar10.load_data()

    X_cats = X_train[(y_train==3).flatten()]
    X_dogs = X_train[(y_train==5).flatten()]
    X_train = np.vstack((X_cats,X_dogs))

    X_train = X_train/127.5 -1
    y_train=y_train.reshape(-1,1)

    valid = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))

    for epoch in range(epochs):
      idx = np.random.randint(0,X_train.shape[0],batch_size)
      imgs = X_train[idx]
      masked_imgs,missing_parts,_ = self.mask_random(imgs)
      generated_img = self.generator.predict(masked_imgs)

      d_loss_real = self.discriminator.train_on_batch(missing_parts,valid)
      d_loss_fake = self.discriminator.train_on_batch(generated_img,fake)
      d_loss = 0.5*np.add(d_loss_real,d_loss_fake)

      g_loss = self.combined.train_on_batch(masked_imgs,[missing_parts,valid])
      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f , mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0],g_loss[1]))
      if epoch % 1000 == 0:
        fid_score = self.calculate_fid(X_train[np.random.choice(range(len(X_train)), 2)], generated_img)
        print(f'FID score: {fid_score}')
      if epoch % sample_interval == 0:
        idx = np.random.randint(0,X_train.shape[0],6)
        imgs = X_train[idx]
        self.sample_images(epoch,imgs)

  def sample_images(self,epoch,imgs):
    r,c = 3,6
    masked_imgs,missing_parts,(y1,y2,x1,x2) = self.mask_random(imgs)
    generated_img = self.generator.predict(masked_imgs)

    imgs = 0.5*imgs + 0.5
    masked_imgs = 0.5*masked_imgs + 0.5
    generated_img = 0.5*generated_img + 0.5

    fig, axs = plt.subplots(r,c)
    for i in range(c):
      axs[0,i].imshow(imgs[i,:,:])
      axs[0,i].axis('off')
      axs[1,i].imshow(masked_imgs[i,:,:])
      axs[1,i].axis('off')
      filled_im=imgs[i].copy()
      filled_im[y1[i]:y2[i],x1[i]:x2[i]]=generated_img[i]
      axs[2,i].imshow(filled_im)
      axs[2,i].axis('off')
    plt.savefig('inpainting_images/%d.png' % epoch)
    plt.close()

  def save_model(self):
    def save(model, model_name):
      model_path = "saved_model/%s.json" % model_name
      weights_path = "saved_model/%s.weights.h5" % model_name
      options = {"file_arch": model_path,"file_weight": weights_path}
      json_string = model.to_json()
      open(options['file_arch'], 'w').write(json_string)
      model.save_weights(options['file_weight'])
    save(self.generator, "generator")
    save(self.discriminator, "discriminator")




In [11]:
if __name__ == '__main__':
  inpainting = Inpainting()

  inpainting.train(epochs=35000,batch_size=128,sample_interval=1000)
  inpainting.save_model()


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 3ms/step
32504 [D loss: 0.000009, acc.: 100.00%] [G loss: 0.084381 , mse: 0.083081]
4/4 [==============================] - 0s 3ms/step
32505 [D loss: 0.000013, acc.: 100.00%] [G loss: 0.076620 , mse: 0.075101]
4/4 [==============================] - 0s 3ms/step
32506 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.089170 , mse: 0.087879]
4/4 [==============================] - 0s 3ms/step
32507 [D loss: 0.000583, acc.: 100.00%] [G loss: 0.081389 , mse: 0.079354]
4/4 [==============================] - 0s 3ms/step
32508 [D loss: 0.000010, acc.: 100.00%] [G loss: 0.091716 , mse: 0.090541]
4/4 [==============================] - 0s 3ms/step
32509 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.087717 , mse: 0.086885]
4/4 [==============================] - 0s 3ms/step
32510 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.087071 , mse: 0.085821]
4/4 [==============================] - 0s 3ms/step
32511 [D loss: 0